# Variant Calling : Generating a standard VCF
```
pi:ababaian
start: 2016 11 21
complete : ... unfinished
```
## Introduction

To be able to do variant analysis to any reasonable quanity (i.e. hundreds of genomes/transcriptomes) and then do comparisons it's going to be important to learn to use the VCF file format and do cross-sample comparisons of VCF-based variants.

Essentially the VCF standard is a way of describing what reads look like at a given position in the genome. I should be able to parse this format.

In general the workflow should look something like this.

    1. Download SRA / FASTQ / BAM file
    2. Align to hgr.fa or a variant  hgr.fa genome (not made yet)
    3. Extract the aligned reads in the transcribed, 45s region from the genome. Possibly break this down further into 18S, 5.8S, 5S, 28S regions seperately.
    4. Generate a VCF file for the extracted region containing 'variants' only.
    5. Plot/analyze many sets of variants side-by-side
    
This is less hypothesis driven; more an exploration of what's available.

## Objective

- Define a variant calling standard which is compatible with calling varients over rDNA aligned sequences for

A) Illumina DNA sequencing
B) PacBio DNA sequencing
C) Illumina RNA sequencing


## Materials and Methods

#### Set-up
Pilot Genome: `$CROWN/data/1kgenomes/NA19240_hgr.bam`
(NA19240 DNAseq Illumina HiSeq2000 paired-end)


##### Methods to test
- bftools
- samtools mpileup
- gatk
- lumpy

In [2]:
cd ~/Crown/data/1kgenomes

## BCFtools / samtools

In [1]:
bcftools --version
echo ''
samtools --version

bcftools 1.3.1
Using htslib 1.3.1
Copyright (C) 2016 Genome Research Ltd.
License Expat: The MIT/Expat license
This is free software: you are free to change and redistribute it.
There is NO WARRANTY, to the extent permitted by law.

samtools 1.3.1
Using htslib 1.3.1
Copyright (C) 2016 Genome Research Ltd.


In [ ]:
# Initial Run

## Basic pile-up

## Basic pile-up; depth 1000 
samtools mpileup --max-depth 1000 -f ~/Crown/resources/hgr/hgr.fa \
-O -s -o na19240.run1.vcf NA19240_hgr.bam

## Basic pile-up; depth 2000
samtools mpileup --max-depth 1000 -f ~/Crown/resources/hgr/hgr.fa \
-O -s -o na19240.run1.vcf NA19240_hgr.bam

## No difference in output files

In [6]:
## This gave me a pretty easy way to look at variant positions
## but it's not a 'BCF' file or compatiable with variant calling
## This would be a good way to plot single positions
## i.e.  chr13   1003635

samtools mpileup -r chr13:1003634-1003635 --max-depth 1000 -f ~/Crown/resources/hgr/hgr.fa \
-O -s NA19240_hgr.bam

# 1003634 is covered by 185 bases, all match the reference G
# 1003635 is covered by 183 bases, all mismatch the reference G for a C
#
# Note all the reads are on the "Forward Strand", downstream of this
# there are more variants which likely preclude mapping from the
# reverse strand


[mpileup] 1 samples in 1 input files
<mpileup> Set max per-file depth to 8000
chr13	1003634	G	185	.$........................................................................................................................................................................................	8DDc<B_DDEDD@e8D8DcBD\BeDDDDeDBDDcODV^aDDDeDFBDDBXD^6DDD6BDBDDDDDDD@PDDBdD[DFFFFFFJFIJIJAEGGJdD@GJGJiIGJJJJJGJIJJBIJgJGGJJIEJGIIJgJIIIJJIJ0IJGIGGGHGGH?HHHH^HHDHH@FFDFF>FFF@FD=;;<<;;;<;;	KKKKKKKKKKKKKKKKKKKKKKKKKKKKIKKKKKKKKKKKKKKK9KKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKIKKKKKKKKKKKKKKKKKIKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKIKKKKKKKKKKKKKKKKKKKKKKKKKKKKK	100,71,64,58,52,51,51,50,49,49,49,49,49,48,48,47,46,46,46,45,44,44,44,44,43,43,43,43,42,42,41,41,40,40,39,37,38,38,38,36,36,36,34,35,35,35,34,35,33,34,34,34,34,33,33,33,31,31,31,30,30,29,29,29,29,29,29,29,29,29,28,28,26,26,26,26,25,25,25,25,25,25,23,23,23,22,22,22,22,22,22,22,22,21,21,21,21,21,21,21,21,21,21,21,20,20,20,20,19,19,18,18,18,18,

In [9]:
## bctTools

samtools mpileup -uf ~/Crown/resources/hgr/hgr.fa -r chr13:1003600-1003700 \
NA19240_hgr.bam | bcftools call -mv -
echo ==============================================================================
samtools mpileup -uf ~/Crown/resources/hgr/hgr.fa -r chr13:1003600-1003700 \
NA19240_hgr.bam | bcftools call -mv --ploidy-file hgr.ploidy -

# where hgr.ploidy is
# chr13 1 1060000 F 100

## Only works with ploidy 0, 1 ,2 ==> not applicable here
## 
## With that assumption here's the call
#
# CHROM	POS	ID	REF	ALT	QUAL	FILTER	INFO	FORMAT	NA19240_hgr.bam
# chr13	1003635	.	G	C	228	.	DP=234;VDB=3.42646e-25;SGB=-0.693147;MQ0F=0;AC=2;AN=2;DP4=0,0,183,0;MQ=41	GT:PL	1/1:255,255,0

Note: Neither --ploidy nor --ploidy-file given, assuming all sites are diploid
[mpileup] 1 samples in 1 input files
<mpileup> Set max per-file depth to 8000
##fileformat=VCFv4.2
##FILTER=<ID=PASS,Description="All filters passed">
##samtoolsVersion=1.3.1+htslib-1.3.1
##samtoolsCommand=samtools mpileup -uf /home/artem/Crown/resources/hgr/hgr.fa -r chr13:1003600-1003700 NA19240_hgr.bam
##reference=file:///home/artem/Crown/resources/hgr/hgr.fa
##contig=<ID=chr13,length=1053000>
##ALT=<ID=*,Description="Represents allele(s) other than observed.">
##INFO=<ID=INDEL,Number=0,Type=Flag,Description="Indicates that the variant is an INDEL.">
##INFO=<ID=IDV,Number=1,Type=Integer,Description="Maximum number of reads supporting an indel">
##INFO=<ID=IMF,Number=1,Type=Float,Description="Maximum fraction of reads supporting an indel">
##INFO=<ID=DP,Number=1,Type=Integer,Description="Raw read depth">
##INFO=<ID=VDB,Number=1,Type=Float,Description="Variant Distance Bias for filtering splice-site artefac

## GATK
(Not Finished)
#### Installation
GATK v3.6-0-g89b7209

- Downloaded the pre-compiled java file
- On local computer; copied `GenomeAnalysisTK.jar` to /usr/local/bioinf/gatk/
- Wrote a short wrapper script into `/usr/local/bin/gatk`
~~~
#!/bin/zsh

echo "Running GATK"
echo "java -Xmx4G -jar /usr/local/bioinf/gatk/GenomeAnalysisTK.jar $@"
java -Xmx4G -jar /usr/local/bioinf/gatk/GenomeAnalysisTK.jar $@
~~~

#### Picard Installation
[Picard v2.2.7](https://github.com/broadinstitute/picard/releases/tag/2.7.1)
- wrapper script into `/usr/local/bin/picard`
- chmod 777 picard/gatk files
~~~
#!/bin/zsh

java -Xmx4G -jar /usr/local/bioinf/picard.jar $@
~~~                                          


#### Running GATK

From: Moreira et al., 2016 [doi: 10.1093/nar/gkw188](https://www.ncbi.nlm.nih.gov/pubmed/27001515) who did mt-rDNA variant calling in *Diplonema sp.* and *Rhynchopus sp.* using illumina PE reads.

*" Variant sites in mtDNA were determined with the UnifiedGenotyper module of the Genome Analysis Toolkit (GATK) v3.3.1 (https://www.broadinstitute.org/gatk/ (45,46)) and FreeBayes (Garrison E and Marth G. (2012) In arXiv (ed.), Vol. 1207.3907v2 [q-bio.GN]). For both
tools we set the ploidy to 100, the minimum number of observed variants to 2, the minimum base quality and mapping quality to 30 and the minimum allele frequency to 0.01. The output files of DNA–DNA comparison is referred to as DDd.vcf files. Only sites with at least 10%
allele frequency were considered. We validated the obtained genome variants by visual inspection with IGV v2.3.40 (https://www.broadinstitute.org/igv/) (39), as well as with
reads generated by Sanger and 454-FLX (Roche). For linked sites, we consolidated the allele frequencies reported by the variant caller software by calculating the mean across all linked sites. "*

Reading GATK documentation there are two main options to consider. The above 'UnifiedCaller' is deprecated.

* HaplotypeCaller: Call DNAseq variants with local re-alignments (ref?)
* MuTect2: Call variants from Normal:Tumour pairs (consider with RNAseq where this data is available). Can only be used for single pairs of samples.


In [20]:
# GATK index HGR genome
# hgr.gatk.fa created with removing all 'empty' chromosomes
# to get GATK to run
#
cd ~/Crown/resources/hgr
    picard CreateSequenceDictionary R=hgr.gatk.fa O=hgr.gatk.dict
    samtools faidx hgr.gatk.fa

[Tue Nov 22 17:47:54 PST 2016] picard.sam.CreateSequenceDictionary REFERENCE=hgr.gatk.fa OUTPUT=hgr.gatk.dict    TRUNCATE_NAMES_AT_WHITESPACE=true NUM_SEQUENCES=2147483647 VERBOSITY=INFO QUIET=false VALIDATION_STRINGENCY=STRICT COMPRESSION_LEVEL=5 MAX_RECORDS_IN_RAM=500000 CREATE_INDEX=false CREATE_MD5_FILE=false GA4GH_CLIENT_SECRETS=client_secrets.json
[Tue Nov 22 17:47:54 PST 2016] Executing as artem@glitch on Linux 4.4.0-45-generic amd64; OpenJDK 64-Bit Server VM 1.8.0_91-8u91-b14-3ubuntu1~16.04.1-b14; Picard version: 2.7.1-SNAPSHOT
[Tue Nov 22 17:47:55 PST 2016] picard.sam.CreateSequenceDictionary done. Elapsed time: 0.00 minutes.
Runtime.totalMemory()=253231104


In [25]:
# Write Read-Groups to NA19240_hgr.bam

picard AddOrReplaceReadGroups \
I=NA19240_hgr.bam O=NA19240_hgr.rg.bam \
RGID=1 \
RGLB=SRR794330 \
RGSM=1kg_na19240 \
RGPL=illumina \
RGPU=unit1

[Tue Nov 22 18:01:14 PST 2016] picard.sam.AddOrReplaceReadGroups INPUT=NA19240_hgr.bam OUTPUT=NA1940_hgr.rg.bam RGID=1 RGLB=SRR794330 RGPL=illumina RGPU=unit1 RGSM=1kg_na19240    VERBOSITY=INFO QUIET=false VALIDATION_STRINGENCY=STRICT COMPRESSION_LEVEL=5 MAX_RECORDS_IN_RAM=500000 CREATE_INDEX=false CREATE_MD5_FILE=false GA4GH_CLIENT_SECRETS=client_secrets.json
[Tue Nov 22 18:01:14 PST 2016] Executing as artem@glitch on Linux 4.4.0-45-generic amd64; OpenJDK 64-Bit Server VM 1.8.0_91-8u91-b14-3ubuntu1~16.04.1-b14; Picard version: 2.7.1-SNAPSHOT
INFO	2016-11-22 18:01:14	AddOrReplaceReadGroups	Created read group ID=1 PL=illumina LB=SRR794330 SM=1kg_na19240

INFO	2016-11-22 18:01:50	AddOrReplaceReadGroups	Processed     1,000,000 records.  Elapsed time: 00:00:35s.  Time for last 1,000,000:   35s.  Last read position: */*
INFO	2016-11-22 18:02:16	AddOrReplaceReadGroups	Processed     2,000,000 records.  Elapsed time: 00:01:02s.  Time for last 1,000,000:   26s.  Last read position: */*
INFO	201

In [29]:
## Index rg file
samtools index NA19240_hgr.rg.bam

In [3]:
## Simple run gatk: HaplotypeCaller
cd ~/Crown/data/1kgenomes/

gatk -R ~/Crown/resources/hgr/hgr.gatk.fa \
-T HaplotypeCaller \
-I NA19240_hgr.rg.bam \
-o hc_run1.vcf

Running GATK
java -Xmx4G -jar /usr/local/bioinf/gatk/GenomeAnalysisTK.jar -R /home/artem/Crown/resources/hgr/hgr.gatk.fa -T HaplotypeCaller -I NA19240_hgr.rg.bam -o hc_run1.vcf
INFO  15:54:00,186 HelpFormatter - -------------------------------------------------------------------------------- 
INFO  15:54:00,189 HelpFormatter - The Genome Analysis Toolkit (GATK) v3.6-0-g89b7209, Compiled 2016/06/01 22:27:29 
INFO  15:54:00,189 HelpFormatter - Copyright (c) 2010-2016 The Broad Institute 
INFO  15:54:00,189 HelpFormatter - For support and documentation go to https://www.broadinstitute.org/gatk 
INFO  15:54:00,190 HelpFormatter - [Thu Nov 24 15:54:00 PST 2016] Executing on Linux 4.4.0-45-generic amd64 
INFO  15:54:00,190 HelpFormatter - OpenJDK 64-Bit Server VM 1.8.0_111-8u111-b14-2ubuntu0.16.04.2-b14 JdkDeflater 
INFO  15:54:00,195 HelpFormatter - Program Args: -R /home/artem/Crown/resources/hgr/hgr.gatk.fa -T HaplotypeCaller -I NA19240_hgr.rg.bam -o hc_run1.vcf 
INFO  15:54:00,201 HelpFo

In [1]:
## The output file looks good but there is quite a bit of improvement to be made
## for rDNA variant calling
cd ~/Crown/data/1kgenomes/
head -n80 hc_run1.vcf # start of vcf file (18s is 3514 - 5772)

##fileformat=VCFv4.2
##FILTER=<ID=LowQual,Description="Low quality">
##FORMAT=<ID=AD,Number=R,Type=Integer,Description="Allelic depths for the ref and alt alleles in the order listed">
##FORMAT=<ID=DP,Number=1,Type=Integer,Description="Approximate read depth (reads with MQ=255 or with bad mates are filtered)">
##FORMAT=<ID=GQ,Number=1,Type=Integer,Description="Genotype Quality">
##FORMAT=<ID=GT,Number=1,Type=String,Description="Genotype">
##FORMAT=<ID=PL,Number=G,Type=Integer,Description="Normalized, Phred-scaled likelihoods for genotypes as defined in the VCF specification">
##GATKCommandLine.HaplotypeCaller=<ID=HaplotypeCaller,Version=3.6-0-g89b7209,Date="Tue Nov 22 19:26:32 PST 2016",Epoch=1479871592455,CommandLineOptions="analysis_type=HaplotypeCaller input_file=[NA19240_hgr.rg.bam] showFullBamList=false read_buffer_size=null read_filter=[] disable_read_filter=[] intervals=null excludeIntervals=null interval_set_rule=UNION interval_merging=ALL interval_padding=0 reference_sequence=

#### Primary corrections of vcf file

`##INFO=<ID=AC,Number=A,Type=Integer,Description="Allele count in genotypes, for each ALT allele, in the same order as listed">`
- The number of alternative alleles called is 1 (A), allow for multiple alternative alleles to be called (R, multiple or G, all alleles or ., uknown)

- No variant call for 3,838. Minor allele frequency (within sample) is 5%. Why wasn't this called?
- It's within a 'CpG' --> many of the reads with 'A' allele also have other neighboring points mutations
`
chr13:1,003,838
<hr>Total count: 652
A      : 34  (5%,     15+,   19- )
C      : 617  (95%,     283+,   334- )
G      : 0
T      : 1  (0%,     0+,   1- )
N      : 0
---------------
`

In [2]:
# Second run gatk: HaplotypeCaller
cd ~/Crown/data/1kgenomes/

# Commands to keep in mind
# -rf <Read Filters to apply>

# Commands of relevance for rDNA
# -L , --interval. Limit to 45s
# -L chr13:1000000-1013500 

# --activityProfileOut <file>. Raw activity profile in IGV format

gatk -R ~/Crown/resources/hgr/hgr.gatk.fa \
-T HaplotypeCaller \
-I NA19240_hgr.rg.bam \
#-o hc_run1.vcf

Running GATK
java -Xmx4G -jar /usr/local/bioinf/gatk/GenomeAnalysisTK.jar -R /home/artem/Crown/resources/hgr/hgr.gatk.fa -T HaplotypeCaller -I NA19240_hgr.rg.bam -o hc_run1.vcf
INFO  15:53:32,054 HelpFormatter - -------------------------------------------------------------------------------- 
INFO  15:53:32,056 HelpFormatter - The Genome Analysis Toolkit (GATK) v3.6-0-g89b7209, Compiled 2016/06/01 22:27:29 
INFO  15:53:32,056 HelpFormatter - Copyright (c) 2010-2016 The Broad Institute 
INFO  15:53:32,056 HelpFormatter - For support and documentation go to https://www.broadinstitute.org/gatk 
INFO  15:53:32,056 HelpFormatter - [Thu Nov 24 15:53:32 PST 2016] Executing on Linux 4.4.0-45-generic amd64 
INFO  15:53:32,056 HelpFormatter - OpenJDK 64-Bit Server VM 1.8.0_111-8u111-b14-2ubuntu0.16.04.2-b14 JdkDeflater 
INFO  15:53:32,060 HelpFormatter - Program Args: -R /home/artem/Crown/resources/hgr/hgr.gatk.fa -T HaplotypeCaller -I NA19240_hgr.rg.bam -o hc_run1.vcf 
INFO  15:53:32,064 HelpFo

## Lumpy

Lumpy is designed to detect structural variations. I need to test and compare it's performance on *bowtie2* and *stampy* aligned hgr genomes to see which performs better.

#### Installation
```
git clone --recursive https://github.com/arq5x/lumpy-sv.git
cd lumpy-sv
make
sudo cp bin/* /usr/local/bin/
cd ..
sudo mv lumpy-sv /usr/local/bioinf/
```

#### Pre-processing data for Lumpy
Almost exactly as from the Lumpy manual:
```
# Extract the discordant paired-end alignments.
samtools view -b -F 1294 NA19240_hgr.bam > NA19240_discord.unsorted.bam

# Extract the split-read alignments
samtools view -h NA19240_hgr.bam \
    | /usr/local/bioinf/lumpy-sv/scripts/extractSplitReads_BwaMem -i stdin \
    | samtools view -Sb - \
    > NA19240_splitters.unsorted.bam

# Sort both alignments
samtools sort NA19240_discord.unsorted.bam -o NA19240_discord.bam
samtools sort NA19240_splitters.unsorted.bam -o NA19240_splitters.bam
```

Extract Insert-size analysis from data
```
samtools view NA19240_hgr.bam \
    | tail -n+100000 \
    | /usr/local/bioinf/lumpy-sv/scripts/pairend_distro.py \
    -r 101 \
    -X 4 \
    -N 10000 \
    -o NA19240_hgr.histo
```
> Removed 53 outliers with isize >= 638

> mean:173.499547602	stdev:72.9427287375

#### Running Lumpy on bt2 aligned data
```
lumpy -mw 4 -tt 0 \
  -pe id:na19240_bt2,bam_file:NA19240_discord.bam,histo_file:NA19240_hgr.histo,mean:173.5,stdev:72.94,read_length:100,min_non_overlap:100,discordant_z:5,back_distance:10,weight:1,min_mapping_threshold:20 \
  > NA19240_lumpy.vcf
```

```
lumpy -mw 4 -tt 0 \
  -pe id:na19240_bt2,bam_file:NA19240_discord.bam,histo_file:NA19240_hgr.histo,mean:173.5,stdev:72.94,read_length:100,min_non_overlap:100,discordant_z:5,back_distance:10,weight:1,min_mapping_threshold:20 \
  > NA19240_stumpy.vcf
```

In [4]:
cat NA19240_lumpy.vcf

##fileformat=VCFv4.2
##source=LUMPY
##INFO=<ID=SVTYPE,Number=1,Type=String,Description="Type of structural variant">
##INFO=<ID=SVLEN,Number=.,Type=Integer,Description="Difference in length between REF and ALT alleles">
##INFO=<ID=END,Number=1,Type=Integer,Description="End position of the variant described in this record">
##INFO=<ID=STRANDS,Number=.,Type=String,Description="Strand orientation of the adjacency in BEDPE format (DEL:+-, DUP:-+, INV:++/--)">
##INFO=<ID=IMPRECISE,Number=0,Type=Flag,Description="Imprecise structural variation">
##INFO=<ID=CIPOS,Number=2,Type=Integer,Description="Confidence interval around POS for imprecise variants">
##INFO=<ID=CIEND,Number=2,Type=Integer,Description="Confidence interval around END for imprecise variants">
##INFO=<ID=CIPOS95,Number=2,Type=Integer,Description="Confidence interval (95%) around POS for imprecise variants">
##INFO=<ID=CIEND95,Number=2,Type=Integer,Description="Confidence interval (95%) around END for imprecise variants">
##INFO

In [5]:
# Data was re-pre-proccessed using Stampy alignment as input.
cat NA19240_stumpy.vcf

##fileformat=VCFv4.2
##source=LUMPY
##INFO=<ID=SVTYPE,Number=1,Type=String,Description="Type of structural variant">
##INFO=<ID=SVLEN,Number=.,Type=Integer,Description="Difference in length between REF and ALT alleles">
##INFO=<ID=END,Number=1,Type=Integer,Description="End position of the variant described in this record">
##INFO=<ID=STRANDS,Number=.,Type=String,Description="Strand orientation of the adjacency in BEDPE format (DEL:+-, DUP:-+, INV:++/--)">
##INFO=<ID=IMPRECISE,Number=0,Type=Flag,Description="Imprecise structural variation">
##INFO=<ID=CIPOS,Number=2,Type=Integer,Description="Confidence interval around POS for imprecise variants">
##INFO=<ID=CIEND,Number=2,Type=Integer,Description="Confidence interval around END for imprecise variants">
##INFO=<ID=CIPOS95,Number=2,Type=Integer,Description="Confidence interval (95%) around POS for imprecise variants">
##INFO=<ID=CIEND95,Number=2,Type=Integer,Description="Confidence interval (95%) around END for imprecise variants">
##INFO

## Discussion

### Structural Variation Calling
* Overall no structural variants were called in 18S / 5.8S / 28S. This may be due to no 'split reads' being generated in the pre-processing script which means smaller indels are not being detected (like observed in the expansion loops)
* In the non-transcribed spacers, there is a huge amount of structural and non-structural variation which is difficult to distinguish from sequencing artifact.